In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import FinalStreamingStdOutCallbackHandler

chat  = ChatOpenAI(
  temperature=0.1,
  streaming=True, 
  callbacks=[FinalStreamingStdOutCallbackHandler()]
)

template = PromptTemplate.from_template('What is the distance between {country_a} and {country_b} Also, what is your name?')
prompt = template.format(country_a='Mexico', country_b='Thailand')

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers). My name is Assistant.'

In [3]:

template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography export. And you only reply in {language}."), 
  ("ai", "Ciao, mi chiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b} Also, what is your name?")
])

prompt = template.format_messages(
  language='Greek',
  name="Socrates",
  country_a='Mexico',
  country_b='Thailand'
)
chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

from langchain.schema import BaseOutputParser 
class CommaOutputParser(BaseOutputParser):
  def parse(self, test):
    items = test.strip().split(',')
    return list(map(str.strip, items))
    

# p = CommaOutputParser()
# p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [6]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase . Do NOT reply with anything else"), 
  ("human", "{question}"),
])

prompt = template.format_messages(
  max_items=10,
  question="What are the planets?"
)
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['mercury', 'venus', 'earth', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune']

In [8]:
chain = template | chat | CommaOutputParser()  # Chain! 
chain.invoke({
  "max_items": 10,
  "question": "What are the pokemons?"
})

['bulbasaur',
 'ivysaur',
 'venusaur',
 'charmander',
 'charmeleon',
 'charizard',
 'squirtle',
 'wartortle',
 'blastoise',
 'pikachu']

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat  = ChatOpenAI(
  temperature=0.1,
  streaming=True, 
  callbacks=[StreamingStdOutCallbackHandler()]
)
chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat 

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
  ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "indian",})

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) diced tomatoes\n- 2 tbsp tomato paste\n- 1 tbsp garam masala\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1 tsp turmeric\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 1/2 cup coconut cream (or any other dairy-free cream alternative)\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the tofu or paneer in the yogurt and spice mixture. Let it marinate for at 